# Execute metagenome functional profile with Paladin
Jacobo de la Cuesta-Zuluaga. June 2025.

The aim of this notebook is to obtain the functional profile from metagenome samples.


## Before we start
This notebook assumes that the sequences already went through QC. In this case, we're using the output files from the `taxprofiler` pipeline, which performs sequence quality control and removal of host sequences. See notebook 01 for that. 

In addition, you need to have a `conda` environment with `paladin` installed. [See their repo here.](https://github.com/ToniWestbrook/paladin)

## Load libraries and set paths

In [1]:
# Libraries
library(tidyverse)
library(conflicted)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Solve conflicts
conflicts_prefer(dplyr::filter)

[conflicted] Will prefer dplyr::filter over any other package.


In [3]:
# Directories
# Base directory
base_dir = "/mnt/lustre/groups/maier/maide581/projects/Metemgee"

# Data
data_dir = file.path(base_dir, "data")

# Sequences
seq_dir = file.path(data_dir, "taxprofiler/analysis_ready_fastqs")

# Out
paladin_dir = file.path(data_dir, "paladin")
dir.create(paladin_dir)

# Paladin output
out_dir = file.path(paladin_dir, "output")
dir.create(out_dir)

# Sheets dir
sheets_dir = file.path(paladin_dir, "sheets")
dir.create(sheets_dir)

# tmp dir
tmp_dir = tempdir()

# paladin index
paladin_index = "/mnt/lustre/groups/maier/databases/Paladin/uhgp-90.faa"

conda_env = "paladin"

Warning message in dir.create(paladin_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/paladin' already exists”
Warning message in dir.create(out_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/paladin/output' already exists”
Warning message in dir.create(sheets_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/paladin/sheets' already exists”


We will use the sequences we have previously processed. These are two quality-controlled samples using the `nf-core/taxprofiler` pipeline. For instructions on how to retrieve and perform QC, see the `01_Run_QC_Taxprofiler.ipynb` notebook

## Execute Paladin

To execute `paladin` we'll need an indexed reference. For general usage with human metagenome samples, we can use the Unified Human Gut Genome (UHGG) protein cataolg. To see how the index was created or create your own, see the notebok in the `Metemgee/helper_scripts/paladin_index` folder

### Create samples file
Similar to the file we passed to taxprofiler, we'll need to create a file with the name of the sample and the files corresponding to forward and reverse reads.

Importantly, this file needs to have a first column called `ArrayTaskID` with the number of the sample (1 for first sample, 2 for second and so on).

**Note** that in this case we'll need the clean reads, not the raw reads.

In [4]:
# Create samples file
# List clean sequences
clean_seq_list = list.files(seq_dir,  
        pattern = "1.merged.fastq.gz",
        full.names = TRUE)

# Combine lists of files to create a data frame
reads_df = data.frame(Forward = clean_seq_list) %>%
    mutate(Sample_name = basename(Forward), # Sample name from the file
        Sample_name = str_remove(Sample_name, "_[0-9]\\.merged.*"),
        ArrayTaskID = row_number()) %>%
    relocate(ArrayTaskID, Sample_name, Forward) # Reorder columns

reads_df %>%
    head()

,ArrayTaskID,Sample_name,Forward
,<int>,<chr>,<chr>
1,1,MI-142-H,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/taxprofiler/analysis_ready_fastqs/MI-142-H_1.merged.fastq.gz
2,2,MI-237-H,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/taxprofiler/analysis_ready_fastqs/MI-237-H_1.merged.fastq.gz


In [5]:
# Write samples file
paladin_samplesfile = file.path(data_dir, "samples_file_paladin.tsv")
write_tsv(reads_df,
    file = paladin_samplesfile)

In [6]:
paladin_array_slurm_raw = str_glue(.open = "[", .close = "]",
"#!/bin/bash
##############################
#       Parameters           #
##############################

# This section will tell the cluster what are the resources your job will need.
# Change the parameters accordingly and carefully!
# The parameters here are a sensible start.

# Name of the job
#SBATCH --job-name=[[job_name]]

# Generate an output file and give it a name
#SBATCH --output=%x-%j.out

# Number of tasks
#SBATCH --ntasks=1

# Number of cpus that this task will need
#SBATCH --cpus-per-task=[[cpu]]

# Specify the total memory required per node
#SBATCH --mem=[[mem]]

# Specify the maximum time this job can take to run before being killed (hh:mm:ss)
#SBATCH --time=23:59:59

# Specify number of array jobs
#SBATCH --array=[[array_jobs]]

# job information
scontrol show job ${SLURM_JOB_ID}

# per node
# prep
source $HOME/.bashrc

# Specify the path to the config file
config=[[samples_file]]

# Extract the sample name for the current $SLURM_ARRAY_TASK_ID
sample=$(awk -v ArrayTaskID=$SLURM_ARRAY_TASK_ID '$1==ArrayTaskID {print $2}' $config)

# Extract the path to the forward read for the current $SLURM_ARRAY_TASK_ID
forward=$(awk -v ArrayTaskID=$SLURM_ARRAY_TASK_ID '$1==ArrayTaskID {print $3}' $config)

# Print to a file a message that includes the current $SLURM_ARRAY_TASK_ID and sample name
echo This is array task ${SLURM_ARRAY_TASK_ID}, the sample name is ${sample} the forward read is ${forward}

# do your real computation
# Activate conda
conda activate [[conda_env]]
cd [[out_dir]]

# Create tmp dir
base_tmp='[[tmp_dir]]'
tmp_dir=${base_tmp}/${sample}'_tmp'
mkdir -p ${tmp_dir}

# Execute paladin and create sorted bam file
paladin align -t [[cpu]] [[index]] ${forward} | \
    samtools view -@ [[cpu]] -b - | \
    samtools sort -@ [[cpu]] - > ${tmp_dir}/${sample}.sorted.bam

# Extract counts
samtools index -@ [[64]] ${tmp_dir}/${sample}.sorted.bam
samtools idxstats -@ [[cpu]] ${tmp_dir}/${sample}.sorted.bam > ${sample}.counts
")

In [7]:
paladin_array_slurm = str_glue(paladin_array_slurm_raw,
        job_name = "paladin_array", 
        array_jobs = str_c("1-", nrow(reads_df)), # number of array jobs should be expressed as 1-<number of samples to run>, if 10 samples, 1-10
        samples_file = paladin_samplesfile, # Samples file we created above
        index = paladin_index,
        out_dir = out_dir,
        tmp_dir = tmp_dir,
        cpu = 16,
        mem = "64G",
        conda_env = conda_env, # Name of conda environment, defined above
        .open = "[", .close = "]") 

paladin_array_slurm %>%
        print()

#!/bin/bash
##############################
#       Parameters           #
##############################

# This section will tell the cluster what are the resources your job will need.
# Change the parameters accordingly and carefully!
# The parameters here are a sensible start.

# Name of the job
#SBATCH --job-name=paladin_array

# Generate an output file and give it a name
#SBATCH --output=%x-%j.out

# Number of tasks
#SBATCH --ntasks=1

# Number of cpus that this task will need
#SBATCH --cpus-per-task=16

# Specify the total memory required per node
#SBATCH --mem=64G

# Specify the maximum time this job can take to run before being killed (hh:mm:ss)
#SBATCH --time=23:59:59

# Specify number of array jobs
#SBATCH --array=1-2

# job information
scontrol show job ${SLURM_JOB_ID}

# per node
# prep
source $HOME/.bashrc

# Specify the path to the config file
config=/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/samples_file_paladin.tsv

# Extract the sample name for the curren

In [8]:
# Write file
array_slurmfile = file.path(sheets_dir, "array_slurm.sh")
write_lines(paladin_array_slurm, array_slurmfile)

In [9]:
# Command
str_glue("cd {sheets_dir} && sbatch {slurmfile}",
         out_dir = out_dir,
         slurmfile = array_slurmfile)

cd /mnt/lustre/groups/maier/maide581/projects/Metemgee/data/paladin/sheets && sbatch /mnt/lustre/groups/maier/maide581/projects/Metemgee/data/paladin/sheets/array_slurm.sh